In [1]:

# Cell 1: Install & Setup

import gwaslab as gl
import logging

# Simple logging
logging.basicConfig(
    filename='harmonize_log.txt',
    level=logging.INFO,
    format='%(asctime)s %(message)s'
)

logging.info("Harmonization started")
print("GWASLab ready")

GWASLab ready


In [2]:
# Cell 2: Define your file
your_file = "/home/dawit/Documents/Projects/GalaxyTools/GWASLAB/gwaslab-sample-data/bbj_t2d_hm3_chr7_variants.txt.gz"  # ← CHANGE THIS

# Optional: format and build
format_type = "auto"      # or "plink2", "saige", etc.
given_build = None        # Set to "19" or "38" if you know, else None to guess

print(f"Using file: {your_file}")
logging.info(f"File: {your_file}")

Using file: /home/dawit/Documents/Projects/GalaxyTools/GWASLAB/gwaslab-sample-data/bbj_t2d_hm3_chr7_variants.txt.gz


In [3]:
# Cell 3: Load sumstats
print("Loading sumstats...")

ss = gl.Sumstats(
    your_file,
      snpid="SNPID",
             chrom="CHR",
             pos="POS",
             ea="EA",
             nea="NEA",
             neaf="EAF",
             beta="BETA",
             se="SE",
             p="P",
             n="N",
    verbose=True
)

print("Loaded")
logging.info("Sumstats loaded")

Loading sumstats...
2026/01/10 13:45:12 GWASLab v4.0.4 https://cloufield.github.io/gwaslab/
2026/01/10 13:45:12 (C) 2022-2026, Yunye He, Kamatani Lab, GPL-3.0 license, gwaslab@gmail.com
2026/01/10 13:45:12 Python version: 3.12.12 | packaged by conda-forge | (main, Oct 22 2025, 23:25:55) [GCC 14.3.0]
2026/01/10 13:45:12 Start to initialize gl.Sumstats from file :/home/dawit/Documents/Projects/GalaxyTools/GWASLAB/gwaslab-sample-data/bbj_t2d_hm3_chr7_variants.txt.gz
2026/01/10 13:45:14  -Reading columns          : N,P,CHR,SNPID,BETA,EA,EAF,NEA,SE,POS
2026/01/10 13:45:14  -Renaming columns to      : N,P,CHR,SNPID,BETA,EA,EAF,NEA,SE,POS
2026/01/10 13:45:14  -Current Dataframe shape : 1103020  x  10
2026/01/10 13:45:14  -Initiating a status column: STATUS ...
2026/01/10 13:45:14 #WARNING! Version of genomic coordinates is unknown for homo sapiens (build: 99)...
2026/01/10 13:45:14  -NEAF is specified...
2026/01/10 13:45:14  -Checking if 0<= NEAF <=1 ...
2026/01/10 13:45:15  -Converted NEAF t

In [4]:
# Cell 4: Get genome build (given or guess)
if given_build is None:
    print("Guessing build...")
    ss.infer_build()
    
    # Reliable way: check STATUS code after inference
    # First digit of STATUS: 1 = hg19, 2 = hg38
    status_sample = ss.data['STATUS'].iloc[0] if 'STATUS' in ss.data.columns else None
    if status_sample and str(status_sample).startswith('1'):
        build = 'hg19'
    elif status_sample and str(status_sample).startswith('2'):
        build = 'hg38'
    else:
        build = 'unknown'
    
    print(f"Guessed build (from STATUS): {build}")
    logging.info(f"Guessed build (from STATUS): {build}")
else:
    build = given_build
    print(f"Using given build: {build}")
    logging.info(f"Given build: {build}")

build_short = '19' if '19' in str(build).lower() else '38'
print(f"Using short build code: {build_short}")

Guessing build...
2026/01/10 13:45:15  -Genomic coordinates are based on GRCh37/hg19...
2026/01/10 13:45:15 Start to infer genome build version using hapmap3 SNPs ...(v4.0.4)
2026/01/10 13:45:15  -Current Dataframe shape : 1103020 x 11 ; Memory usage: 87.32 MB
2026/01/10 13:45:15 #WARNING! Data types were not fixed for : CHR 
2026/01/10 13:45:15 Start to fix chromosome notation (CHR) ...(v4.0.4)
2026/01/10 13:45:15  -Checking CHR data type...
2026/01/10 13:45:15  -Variants with standardized chromosome notation: 1100517
2026/01/10 13:45:15  -Variants with fixable chromosome notations: 2503
2026/01/10 13:45:15  -No unrecognized chromosome notations...
2026/01/10 13:45:15  -Identifying non-autosomal chromosomes : X, Y, and MT ...
2026/01/10 13:45:15  -Identified 2503 variants on sex chromosomes...
2026/01/10 13:45:15  -Standardizing first sex chromosome chromosome notations: X to 23...
2026/01/10 13:45:15 Finished fixing chromosome notation (CHR).
2026/01/10 13:45:15  -Dtype fixes applied

In [5]:
# Cell 5: Choose references (path or keyword)
# Option 1: Use your own paths (uncomment + fill if you have them)
ref_seq   = "/home/dawit/Documents/Projects/GalaxyTools/GWASLAB/gwaslab-sample-data/chr7.fasta.gz"
ref_infer = "/home/dawit/Documents/Projects/GalaxyTools/GWASLAB/gwaslab-sample-data/1kg_eas_hg19.chr7_126253550_128253550.vcf.gz"
ref_rsid  = "/home/dawit/Documents/Projects/GalaxyTools/GWASLAB/gwaslab-sample-data/1kg_dbsnp151_hg19_auto_hm3_chr7_variants.txt.gz"

# Option 2: Use GWASLab keywords (recommended)
ref_seq_keyword   = f"ucsc_genome_hg{build_short}"
ref_infer_keyword = f"1kg_pan_hg{build_short}"          # pan = safest
ref_rsid_keyword  = "1kg_dbsnp151_hg19_auto" if build_short == '19' else None

# Pick path if given, else keyword
ref_seq   = locals().get('ref_seq',   None) or gl.get_path(ref_seq_keyword)
ref_infer = locals().get('ref_infer', None) or gl.get_path(ref_infer_keyword)
ref_rsid  = locals().get('ref_rsid',  None) or (gl.get_path(ref_rsid_keyword) if ref_rsid_keyword else None)

print("References:")
print(f"  seq:   {ref_seq}")
print(f"  infer: {ref_infer}")
if ref_rsid:
    print(f"  rsid:  {ref_rsid}")

logging.info(f"References: seq={ref_seq}, infer={ref_infer}, rsid={ref_rsid}")

References:
  seq:   /home/dawit/Documents/Projects/GalaxyTools/GWASLAB/gwaslab-sample-data/chr7.fasta.gz
  infer: /home/dawit/Documents/Projects/GalaxyTools/GWASLAB/gwaslab-sample-data/1kg_eas_hg19.chr7_126253550_128253550.vcf.gz
  rsid:  /home/dawit/Documents/Projects/GalaxyTools/GWASLAB/gwaslab-sample-data/1kg_dbsnp151_hg19_auto_hm3_chr7_variants.txt.gz


In [6]:
# Cell 6: Download if needed (uncomment once)
# gl.download_ref(ref_seq_keyword)
# gl.download_ref(ref_infer_keyword)
# if ref_rsid_keyword: gl.download_ref(ref_rsid_keyword)

print("Uncomment downloads above if files are missing")

Uncomment downloads above if files are missing


In [7]:
# Cell 7: QC & Standardize
print("Running basic_check...")

ss.basic_check(
    remove_dup=True,
    threads=4,
    verbose=True
)

print("QC done")
logging.info("QC done")

Running basic_check...
2026/01/10 13:45:19 Start to check SNPID/rsID ...(v4.0.4)
2026/01/10 13:45:19  -Checking SNPID data type...
2026/01/10 13:45:19  -Converted datatype for SNPID: object -> string
2026/01/10 13:45:20  -Checking if SNPID contains NA strings :na,NA,Na,Nan,NaN,<NA>,null,NULL,#N/A,#VALUE!,N/A,n/a,missing,...
2026/01/10 13:45:20  -Checking if SNPID is CHR:POS:NEA:EA...(separator: - ,: , _)
2026/01/10 13:45:20 Finished checking SNPID/rsID.
2026/01/10 13:45:20 Start to fix chromosome notation (CHR) ...(v4.0.4)
2026/01/10 13:45:20  -Checking CHR data type...
2026/01/10 13:45:21  -Variants with standardized chromosome notation: 1103020
2026/01/10 13:45:21  -All CHR are already fixed...
2026/01/10 13:45:21 Finished fixing chromosome notation (CHR).
2026/01/10 13:45:21 Start to fix basepair positions (POS) ...(v4.0.4)
2026/01/10 13:45:21  -Trying to convert datatype for POS: int64 -> Int64...
2026/01/10 13:45:21  -Position bound:(0 , 250,000,000)
2026/01/10 13:45:21  -No outli

In [8]:
# Cell 8: Annotate rsIDs (using fast assign_rsid2)
if ref_rsid:
    print("Annotating rsIDs...")
    ss.assign_rsid2(
        path=ref_rsid,
        threads=4,
        overwrite="all"
    )
    print("rsIDs annotated")
    logging.info("rsIDs annotated")
else:
    print("Skipping rsID annotation")
    logging.info("rsID skipped")

Annotating rsIDs...
2026/01/10 13:45:27 Start to assign rsID from reference ...(v4.0.4)
2026/01/10 13:45:27  -Number of threads/cores to use: 4
2026/01/10 13:45:27  -Current Dataframe shape : 1102246 x 11 ; Memory usage: 79.91 MB
2026/01/10 13:45:27  -Determining reference mode: tsv...
2026/01/10 13:45:27  -Using TSV directly for lookup: /home/dawit/Documents/Projects/GalaxyTools/GWASLAB/gwaslab-sample-data/1kg_dbsnp151_hg19_auto_hm3_chr7_variants.txt.gz...
2026/01/10 13:45:27 Start to assign from lookup table ...(v4.0.4)
2026/01/10 13:45:27  -Current Dataframe shape : 1102246 x 12 ; Memory usage: 88.32 MB
2026/01/10 13:45:27 #WARNING! Cannot update qc_status: Sumstats object not found
2026/01/10 13:45:27  -Initialized ALLELE_FLIPPED column
2026/01/10 13:45:27  -Detected allele mode: EA_NEA using EA(EA/ALT) / NEA(NEA/REF)...
2026/01/10 13:45:28  -Loaded 1,099,686 lookup rows...
2026/01/10 13:45:31  -Found 569971 flipped variants in this chunk
2026/01/10 13:45:32  -Updated ALLELE_FLIPPE

In [ ]:
# Cell 9: Harmonize
print("Harmonizing...")

ss.harmonize(
    basic_check=False,          # already done
    ref_seq=ref_seq,
    ref_infer=ref_infer,
    ref_alt_freq="AF",
    ref_rsid_tsv=ref_rsid,
    threads=4,
    remove=False,
    sweep_mode=True,
    verbose=True
)

ss.flip_allele_stats()

print("Harmonization done")
logging.info("Harmonization done")

Harmonizing...
2026/01/10 13:45:33 Start to check if NEA is aligned with reference sequence ...(v4.0.4)
2026/01/10 13:45:33  -Reference genome FASTA file: /home/dawit/Documents/Projects/GalaxyTools/GWASLAB/gwaslab-sample-data/chr7.fasta.gz
2026/01/10 13:45:33    -Loading and building numpy fasta records:
2026/01/10 13:45:34  -Variants allele on given reference sequence :  0
2026/01/10 13:45:34  -Variants flipped :  0
2026/01/10 13:45:34   -Raw Matching rate :  0.00%
2026/01/10 13:45:34 #WARNING! Matching rate is low, please check if the right reference genome is used.
2026/01/10 13:45:34  -Variants inferred reverse_complement :  0
2026/01/10 13:45:34  -Variants inferred reverse_complement_flipped :  0
2026/01/10 13:45:34  -Both allele on genome + unable to distinguish :  0
2026/01/10 13:45:34  -Variants not on given reference sequence :  0
2026/01/10 13:45:34 Finished checking if NEA is aligned with reference sequence.
2026/01/10 13:45:34 Start to adjust statistics based on STATUS code

In [12]:
# Cell 10: Check results
print("Summary:")
ss.summary()


Summary:


{'overview': {'Row_num': '1102246',
  'Column_num': '14',
  'Column_names': 'SNPID,rsID,CHR,POS,EA,NEA,STATUS,EAF,RAF,BETA,SE,P,N,STRAND',
  'Last_checked_time': 'Sat Jan 10 15:44:02 2026',
  'QC and Harmonization': {'basic_check': {'performed': True,
    'last_executed': '2026-01-10 13:45:27',
    'parameters_used': {'remove': False,
     'remove_dup': True,
     'threads': 4,
     'fix_id_kwargs': {},
     'remove_dup_kwargs': {},
     'fix_chr_kwargs': {},
     'fix_pos_kwargs': {},
     'fix_allele_kwargs': {},
     'sanity_check_stats_kwargs': {},
     'consistency_check_kwargs': {},
     'normalize': True,
     'normalize_allele_kwargs': {},
     'verbose': True}},
   'harmonize': {'performed': True,
    'last_executed': '2026-01-10 13:46:03',
    'parameters_used': {'basic_check': False,
     'ref_seq': '/home/dawit/Documents/Projects/GalaxyTools/GWASLAB/gwaslab-sample-data/chr7.fasta.gz',
     'ref_rsid_tsv': '/home/dawit/Documents/Projects/GalaxyTools/GWASLAB/gwaslab-sample-da

In [16]:
print("\nStatus details:")
ss.lookup_status()



Status details:


,Genome_Build,rsID&SNPID,CHR&POS,Stadardize&Normalize,Align,Panlidromic_SNP&Indel,Count,Percentage(%)
1900090,hg19,rsid valid & SNPID valid,CHR valid & POS valid,standardized SNP,Unchecked,Not_palindromic_SNPs,1097862,99.6
1900097,hg19,rsid valid & SNPID valid,CHR valid & POS valid,standardized SNP,Unchecked,Indistinguishable,68,0.01
1900099,hg19,rsid valid & SNPID valid,CHR valid & POS valid,standardized SNP,Unchecked,Unchecked,1002,0.09
1900399,hg19,rsid valid & SNPID valid,CHR valid & POS valid,standardized & normalized indel,Unchecked,Unchecked,752,0.07
1930090,hg19,rsid invalid & SNPID invalid,CHR valid & POS valid,standardized SNP,Unchecked,Not_palindromic_SNPs,1942,0.18
1930097,hg19,rsid invalid & SNPID invalid,CHR valid & POS valid,standardized SNP,Unchecked,Indistinguishable,28,0.0
1930099,hg19,rsid invalid & SNPID invalid,CHR valid & POS valid,standardized SNP,Unchecked,Unchecked,333,0.03
1930399,hg19,rsid invalid & SNPID invalid,CHR valid & POS valid,standardized & normalized indel,Unchecked,Unchecked,259,0.02


In [17]:
# Cell 11: Save cleaned data
output_name = "clean_harmonized"

ss.to_format(
    output_name,
    fmt="ldsc",
    hapmap3=True,
    exclude_hla=True,
    md5sum=True
)

print(f"Saved: {output_name}.ldsc.tsv.gz")
logging.info(f"Saved: {output_name}.ldsc.tsv.gz")
logging.info("Finished")

2026/01/10 15:46:01 Start to convert the output sumstats in:  ldsc  format
2026/01/10 15:46:02 Start to exclude variants in HLA regions ...(v4.0.4)
2026/01/10 15:46:02  -Excluded 2757 variants in HLA region (chr6: 25000000-34000000 )...
2026/01/10 15:46:02  -Filtered out variants: 2757
2026/01/10 15:46:02  -Current Dataframe shape : 1099489 x 14 ; Memory usage: 113.26 MB
2026/01/10 15:46:02 Finished filtering variants.
2026/01/10 15:46:02  -Genomic coordinates are based on GRCh37/hg19...
2026/01/10 15:46:02 Start to extract HapMap3 SNPs ...(v4.0.4)
2026/01/10 15:46:02  -Loading Hapmap3 variants from built-in datasets...
2026/01/10 15:46:09  -rsID will be used for matching...
2026/01/10 15:46:12  -Raw input contains 1089664 Hapmap3 variants based on rsID...
2026/01/10 15:46:12  -Checking if alleles are same...
2026/01/10 15:46:14  -Filtered 0 Hapmap3 variants due to unmatched alleles...
2026/01/10 15:46:14 Finished extracting HapMap3 SNPs.
2026/01/10 15:46:14  -Extract 1089664 variants 